In [1]:
import pandas as pd
from pyproj import CRS
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.sampledata.us_counties import data as counties
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper, GeoJSONDataSource
output_notebook()

Loading BokehJS ...

In [2]:
df = pd.read_csv("Natality, 2016-2019 expanded.txt", sep="\t")

In [3]:
df = df.drop(["County of Residence Code","Notes","Mother's Single Race 6 Code","Total Population","Birth Rate","Female Population","Fertility Rate"], axis=1)

In [4]:
df["County of Residence"]= df["County of Residence"].str.split(" County, IN", n = 1, expand = True)
df["County of Residence"]= df["County of Residence"].str.split(", IN", n = 1, expand = True)
df["% of Total Births"]= df["% of Total Births"].str.split("%", n = 1, expand = True)
df["Mother's Single Race 6"]= df["Mother's Single Race 6"].astype(str)


In [5]:
df.columns = df.columns.astype(str)
df["County of Residence"]= df["County of Residence"].astype(str)
df["Mother's Single Race 6"]=df["Mother's Single Race 6"].astype(str)
df=df.set_index("County of Residence")
df=df.drop(index="nan")

In [6]:
df= df.replace({"nan":"total"})

In [7]:
df = df.replace({"Not Applicable": -1
    , "Suppressed": -2})

In [8]:
df[["Births","% of Total Births","Average Interval Since Last Live Birth (months)","Average Interval Since Last Other Pregnancy Outcome (months)"]]= df[["Births","% of Total Births","Average Interval Since Last Live Birth (months)","Average Interval Since Last Other Pregnancy Outcome (months)"]].astype("float64")



In [9]:
df.dtypes

Mother's Single Race 6                                           object
Births                                                          float64
% of Total Births                                               float64
Average Age of Mother (years)                                   float64
Average OE Gestational Age (weeks)                              float64
Average LMP Gestational Age (weeks)                             float64
Average Birth Weight (grams)                                    float64
Average Pre-pregnancy BMI                                       float64
Average Number of Prenatal Visits                               float64
Average Interval Since Last Live Birth (months)                 float64
Average Interval Since Last Other Pregnancy Outcome (months)    float64
dtype: object

In [10]:
def profileMaker(df):
    df_p = df.pivot_table(columns="Mother's Single Race 6", 
         values=['Births', "Average Age of Mother (years)",
                 "% of Total Births", 
                 "Average Age of Mother (years)",
                 "Average OE Gestational Age (weeks)",
                 "Average LMP Gestational Age (weeks)",
                 "Average Birth Weight (grams)", 
                 "Average Pre-pregnancy BMI", 
                 "Average Number of Prenatal Visits", 
                 "Average Interval Since Last Live Birth (months)",
                 "Average Interval Since Last Other Pregnancy Outcome (months)"
                ])
    return df_p
        
        

In [11]:
Clark = profileMaker(df[df.index == "Clark"])


In [12]:
def profileMaker_2(df):
    d = {}
    for i in df.index.unique():
        df_p = df[df.index==i].pivot_table(columns="Mother's Single Race 6", 
             values=['Births', "Average Age of Mother (years)",
                     "% of Total Births", 
                     "Average Age of Mother (years)",
                     "Average OE Gestational Age (weeks)",
                     "Average LMP Gestational Age (weeks)",
                     "Average Birth Weight (grams)", 
                     "Average Pre-pregnancy BMI", 
                     "Average Number of Prenatal Visits", 
                     "Average Interval Since Last Live Birth (months)",
                     "Average Interval Since Last Other Pregnancy Outcome (months)"
                    ])
        d[i] = df_p
        
    return d
        
        

In [13]:
p = profileMaker_2(df)
p["Allen"]

Mother's Single Race 6,American Indian or Alaska Native,Asian,Black or African American,More than one race,Native Hawaiian or Other Pacific Islander,Not Available,Not Reported,Unknown or Not Stated,White,total
% of Total Births,-2.00,0.38,1.07,0.17,-2.00,-2.0,-2.0,-2.0,4.93,6.56
Average Age of Mother (years),27.00,27.98,26.24,24.61,32.80,0.0,0.0,0.0,28.40,27.93
Average Birth Weight (grams),3423.50,3103.22,3045.98,3174.07,3543.80,0.0,0.0,0.0,3301.30,3245.26
Average Interval Since Last Live Birth (months),15.50,46.17,46.73,36.87,111.20,0.0,0.0,0.0,42.45,43.37
Average Interval Since Last Other Pregnancy Outcome (months),20.00,42.63,46.15,64.58,55.00,0.0,0.0,0.0,47.14,47.04
Average LMP Gestational Age (weeks),38.75,38.40,38.01,38.51,37.80,0.0,0.0,0.0,38.58,38.47
Average Number of Prenatal Visits,11.75,10.44,10.83,10.17,11.80,0.0,0.0,0.0,11.64,11.40
Average OE Gestational Age (weeks),38.50,38.48,38.06,38.37,38.80,0.0,0.0,0.0,38.54,38.45
Average Pre-pregnancy BMI,25.85,24.81,29.59,28.29,32.46,0.0,0.0,0.0,27.87,27.99
Births,-2.00,310.00,864.00,134.00,-2.00,-2.0,-2.0,-2.0,3985.00,5302.00


In [900]:
df_pp = df.pivot(columns=["Mother's Single Race 6"],
                       values=['Births', 
                               "Average Age of Mother (years)",
                               "% of Total Births", 
                               "Average OE Gestational Age (weeks)",
                               "Average LMP Gestational Age (weeks)",
                               "Average Birth Weight (grams)", 
                               "Average Pre-pregnancy BMI", 
                               "Average Number of Prenatal Visits", 
                               "Average Interval Since Last Live Birth (months)",
                               "Average Interval Since Last Other Pregnancy Outcome (months)"
                              ])

In [771]:
df_pp.columns = df.columns.str.replace(' ', '') 

ValueError: Length mismatch: Expected axis has 110 elements, new values have 11 elements

In [901]:
df_pp.columns = df_pp.columns.map('_'.join)


In [904]:
df_pp.index = df_pp.index.str.upper()

In [905]:
df_pp.index = df_pp.index.rename("COUNTIES")
df_pp= df_pp.reset_index()


In [791]:
#df_pp.to_json("Preg_outcome.json")

In [906]:
#df_pp.columns[df_pp.columns.duplicated()]

In [907]:
#df_pp.drop(['Average Age of Mother (years)_American Indian or Alaska Native',
       'Average Age of Mother (years)_Asian',
       'Average Age of Mother (years)_Black or African American',
       'Average Age of Mother (years)_More than one race',
       'Average Age of Mother (years)_Native Hawaiian or Other Pacific Islander',
       'Average Age of Mother (years)_Not Available',
       'Average Age of Mother (years)_Not Reported',
       'Average Age of Mother (years)_Unknown or Not Stated',
       'Average Age of Mother (years)_White',
       'Average Age of Mother (years)_total'], axis=1)

IndentationError: unexpected indent (<ipython-input-907-ff6b9de7addc>, line 2)

In [908]:
a = gpd.read_file("counties.geojson")
a.shape

(92, 18)

In [956]:
b = df_pp.merge(a, left_on="COUNTIES", right_on="index", how="outer")
b = b.drop(["id","STATEFP","COUNTYFP","COUNTYNS","AFFGEOID","GEOID","LSAD","ALAND","AWATER", "geometry"], axis=1)
b.set_index("index", inplace=True)
b.index = b.index.astype(str)
b.drop(index="nan", inplace=True)
b["COUNTIES"] = b.index.str.capitalize()
for i in df_pp.columns:
    b[i].fillna("No Data", inplace=True)
    
b.to_json("preg_outcome.json", orient="index")

In [959]:
b['Average Age of Mother (years)_total']

index
ALLEN           27.93
CLARK           27.93
DELAWARE        27.02
ELKHART         27.30
HAMILTON        31.18
               ...   
WASHINGTON    No Data
MONTGOMERY    No Data
PERRY         No Data
PIKE          No Data
WHITLEY       No Data
Name: Average Age of Mother (years)_total, Length: 92, dtype: object